# Đồ án 1 - Thu thập dữ liệu

**Danh sách thành viên**

|Họ và tên|MSSV|Công việc|
| :------ | :---: | :--------- |
|Bùi Quang Bảo|19120454|Thu thập dữ liệu bằng API|
|Võ Thành Nam|19120301|Thu thập dữ liệu tracks bằng parse HTML|
|Phạm Lưu Mỹ Phúc|19120331|Thu thập dữ liệu playlists bằng parse HTML|
|Lương Ánh Nguyệt|19120315|Thu thập dữ liệu users bằng parse HTML|

## Import thư viện
Các thư viện được sử dụng trong toàn bộ chương trình.

#### API
* Thư viện requests: tạo HTTP requests, sử dụng để tương tác với API của Soundcloud.
* Thư viện requests_cache: lưu request dưới dạng cache, tiết kiệm thời gian trong quá trình kiểm tra nếu chạy notebook nhiều lần.
* Tạo folder `Api_data` (nếu chưa có) để lưu dữ liệu thu thập được bằng API.

In [1]:
import requests
import json
import pandas as pd
# !pip install requests-cache &> /dev/null
import requests_cache
requests_cache.install_cache('my_cache')
import os
if not os.path.isdir('Api_data'):
    os.makedirs('Api_data')

#### Parse HTML
* Selenium là thư viện chính được sử dụng để lấy dữ liệu từ trang web theo phương thức parse HTML.
* Tạo folder `Crawl_data` (nếu chưa có) để lưu dữ liệu thu thập được bằng parse HTML.

In [2]:
import time
import re
import csv
import urllib.robotparser
from selenium import webdriver
from bs4 import BeautifulSoup
from collections import defaultdict
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
if not os.path.isdir('Crawl_data'):
    os.makedirs('Crawl_data')

## I. Thu thập dữ liệu bằng phương pháp API

Tổng quan: Thu thập dữ liệu user, playlist và track trên trang Soundcloud

Phương pháp: Thông qua Soundcloud API

### Thông tin chung về dữ liệu thu thập được:
* Thành công thu thập dữ liệu, lưu trong các file `user.csv`, `playlist.csv` và `track.csv`, mỗi file có ít nhất 1000 records, thoả mãn yêu cầu đồ án
* Thứ tự thu thập: User -> Playlist -> Track. Dữ liệu sau được thu thập dựa trên dữ liệu trước (playlist của user, track trong playlist)
* Cụ thể hơn sẽ được nêu ra ở từng giai đoạn thu thập dữ liệu

File 'my_cache.sqlite' tạo bởi thư viện requests_cache, lưu tất cả requests, thuận tiện hơn nếu chạy notebook nhiều lần.

### Một số biến toàn cục

Theo như yêu cầu của đồ án, cần thu thập ít nhất 1000 records cho mỗi file output. Để thuận tiện hơn trong quá trình cài đặt và kiểm tra, ta sử dụng biến num_of_entries.

Cliend ID được lấy như sau:
1. Đăng ký/Đăng nhập vào tài khoản Soundcloud
2. F12 để truy cập vào Chrome DevTools
3. Chọn tab Network
4. F5 để reload lại Soundcloud
5. Ở cột "Name", chọn ngẫu nhiên 1 request cho đến khi Request URL (trong tab Headers) có chứa "?client_id="
6. Ta lấy được Client ID từ sau đoạn "?client_id=" cho đến ký tự "?" hoặc "&"

Vì đồ án cũng không yêu cầu cụ thể sẽ thu thập thông tin những user nào, chúng ta sẽ bắt đầu thu thập từ user có id "917161864", username "buiquangbao" (Tài khoản của chính người thực hiện đồ án này, để thuận tiện kiểm tra).

In [3]:
num_of_entries = 1000 # Change to 1000
clientid = '6gsNBd4mJwXr0LxTBh8VKBOrViK6Aj56' # <----- Please login into Soundcloud, find your client ID and change this
init_userid = '917161864'

Chúng ta sẽ tiến hành thu thập dữ liệu theo thứ tự sau:

Users -> Playlists của user đã thu thập được -> Tracks trong playlists thu thập được

### 1. Thu thập dữ liệu "User"

Chúng ta sẽ tiến hành thu thập thông tin của những user có giá trị `playlist_count` khác 0 (có ít nhất 1 public playlist) để đảm bảo khi thu thập dữ liệu playlist, chúng ta không bị thiếu số lượng (có thể số lượng playlist không bằng số lượng user chẳng hạn).

In [4]:
## Request with userid
# req_user = requests.get(f'https://api-v2.soundcloud.com/users/{userid}?client_id={clientid}').json()
## Request with username
# req_user = requests.get(f'https://api-v2.soundcloud.com/resolve?url=https://soundcloud.com/{username}&client_id={clientid}').json()

# Just to get the properties list
req_user = requests.get(f'https://api-v2.soundcloud.com/users/{init_userid}?client_id={clientid}').json()
props_list = list(req_user.keys())

# Create an empty list with the props_list
user_dict = {}
for i in range(len(props_list)):
    user_dict[props_list[i]] = []

# Get ready to request
userid_start = int(init_userid)
count = 0
request_count = 0
continous_fail_count = 0

# Request Soundcloud until we got enough entries
print("Running, please wait...")
while count < num_of_entries:
    userid = str(userid_start)
    req_user = requests.get(f'https://api-v2.soundcloud.com/users/{userid}?client_id={clientid}').json()
    request_count += 1
    is_successful = True

    # print('.', end='')
    
    try:
        temp = req_user['id'] # Just to make sure the request is a user
        continous_fail_count -= 1 if continous_fail_count > 0 else 0
    except:
        is_successful = False
        continous_fail_count += 1
        if continous_fail_count == 10:
            print("Break, failed 10 times continuously!")
            print(f'{request_count} requests was made.')
            break
    
    if is_successful and req_user['playlist_count'] >= 1:
        for i in range(len(props_list)):
            try:
                user_dict[props_list[i]].append(req_user[props_list[i]])
            except:
                print(f'A prop failed at req_user:\n{req_user}')
        count += 1

    userid_start += 3 # Cho tới thời điểm thực hiện đồ án này, quy luật đặt id cho người dùng của Soundcloud là:
                      # Id của user sau = Id của user trước + 3
                      # Thực ra để userid_start += 1 vẫn đúng nhưng tốn thời gian, vì số lượng requests gấp 3 lần

# Just display for testing
df_user = pd.DataFrame(user_dict)
df_user.to_csv(os.path.join('./Api_data', 'user.csv'))

print(f'\n{request_count} requests was made.')
print(df_user.shape)
df_user.head()

Running, please wait...

12776 requests was made.
(1000, 32)


,id,permalink,username,avatar_url,city,comments_count,country_code,created_at,creator_subscriptions,creator_subscription,...,playlist_count,reposts_count,track_count,uri,urn,verified,visuals,badges,station_urn,station_permalink
0,917161864,buiquangbao,Bui Quang Bao,https://i1.sndcdn.com/avatars-BDstqyVzw7DgQ80k...,NaN,0,NaN,2020-12-07T01:53:40Z,[{'product': {'id': 'free'}}],{'product': {'id': 'free'}},...,2,NaN,1,https://api.soundcloud.com/users/917161864,soundcloud:users:917161864,False,"{'urn': 'soundcloud:users:917161864', 'enabled...","{'pro': False, 'pro_unlimited': False, 'verifi...",soundcloud:system-playlists:artist-stations:91...,artist-stations:917161864
1,917161870,user-580689793,RAVER,https://a1.sndcdn.com/images/default_avatar_la...,NaN,0,IT,2020-12-07T01:53:41Z,[{'product': {'id': 'free'}}],{'product': {'id': 'free'}},...,1,NaN,1,https://api.soundcloud.com/users/917161870,soundcloud:users:917161870,False,NaN,"{'pro': False, 'pro_unlimited': False, 'verifi...",soundcloud:system-playlists:artist-stations:91...,artist-stations:917161870
2,917161903,yo-fav-niah,Yo.fav. niah,https://i1.sndcdn.com/avatars-000928304617-t7l...,NaN,0,NaN,2020-12-07T01:53:57Z,[{'product': {'id': 'free'}}],{'product': {'id': 'free'}},...,1,NaN,0,https://api.soundcloud.com/users/917161903,soundcloud:users:917161903,False,NaN,"{'pro': False, 'pro_unlimited': False, 'verifi...",soundcloud:system-playlists:artist-stations:91...,artist-stations:917161903
3,917161906,michael-anthoms,Anthmzs,https://i1.sndcdn.com/avatars-IMhr2nnrY7mamykh...,NaN,0,NaN,2020-12-07T01:53:57Z,[{'product': {'id': 'free'}}],{'product': {'id': 'free'}},...,2,NaN,13,https://api.soundcloud.com/users/917161906,soundcloud:users:917161906,False,NaN,"{'pro': False, 'pro_unlimited': False, 'verifi...",soundcloud:system-playlists:artist-stations:91...,artist-stations:917161906
4,917161924,user-305625325,Jaydaa💕,https://a1.sndcdn.com/images/default_avatar_la...,NaN,0,NaN,2020-12-07T01:54:01Z,[{'product': {'id': 'free'}}],{'product': {'id': 'free'}},...,1,NaN,0,https://api.soundcloud.com/users/917161924,soundcloud:users:917161924,False,NaN,"{'pro': False, 'pro_unlimited': False, 'verifi...",soundcloud:system-playlists:artist-stations:91...,artist-stations:917161924


In [5]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     1000 non-null   int64  
 1   permalink              1000 non-null   object 
 2   username               1000 non-null   object 
 3   avatar_url             1000 non-null   object 
 4   city                   73 non-null     object 
 5   comments_count         1000 non-null   int64  
 6   country_code           80 non-null     object 
 7   created_at             1000 non-null   object 
 8   creator_subscriptions  1000 non-null   object 
 9   creator_subscription   1000 non-null   object 
 10  description            87 non-null     object 
 11  followers_count        1000 non-null   int64  
 12  followings_count       1000 non-null   int64  
 13  first_name             762 non-null    object 
 14  full_name              762 non-null    object 
 15  group

### 2. Thu thập dữ liệu "Playlist"

Với danh sách user, ta sẽ tiến hành thu thập playlist của các user đó.

In [6]:
## Request with playlistid
# req_playlist = requests.get(f'https://api-v2.soundcloud.com/playlists/{playlistid}?client_id={clientid}').json()
## Request with playlisturl
# req_playlist = requests.get(f'https://api-v2.soundcloud.com/resolve?url={playlisturl}&client_id={clientid}').json()
## Request with userid (có thể nhiều playlist)
# req_playlist = requests.get(f'https://api-v2.soundcloud.com/users/{user_id}/playlists?client_id={clientid}').json()

# Just to get the properties list
req_playlist = requests.get(f'https://api-v2.soundcloud.com/users/{init_userid}/playlists?client_id={clientid}').json()
props_list = list(req_playlist['collection'][0].keys())

# Create an empty list with the props_list
playlist_dict = {}
for i in range(len(props_list)):
    playlist_dict[props_list[i]] = []

# Get ready to request
count = 0
request_count = 0

# Request Soundcloud until we got enough entries
print("Running, please wait...")
for user_id in user_dict['id']:
    if count >= num_of_entries:
        break
    req_playlist = requests.get(f'https://api-v2.soundcloud.com/users/{user_id}/playlists?client_id={clientid}').json()
    request_count += 1
    
    # print('.', end='')
    
    for playlist in req_playlist['collection']:
        is_successful = True
        
        try:
            temp = playlist['id']
        except:
            is_successful = False       

        if is_successful:
            for i in range(len(props_list)):
                try:
                    playlist_dict[props_list[i]].append(playlist[props_list[i]])
                except:
                    print(f'A prop failed at playlist:\n{playlist}')
            count += 1

# Just display for testing
df_playlist = pd.DataFrame(playlist_dict)
df_playlist.to_csv(os.path.join('./Api_data', 'playlist.csv'))

print(f'\n{request_count} requests was made.')
print(df_playlist.shape)
df_playlist.head()

Running, please wait...

595 requests was made.
(1001, 33)


,id,title,artwork_url,created_at,description,duration,embeddable_by,genre,kind,label_name,...,tag_list,uri,user_id,set_type,is_album,published_at,display_date,user,tracks,track_count
0,1249674391,Ambience Sound,NaN,2021-04-29T14:00:36Z,NaN,43041945,all,NaN,playlist,NaN,...,NaN,https://api.soundcloud.com/playlists/1249674391,917161864,NaN,False,2021-04-29T14:00:36Z,2021-04-29T14:00:36Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,[{'artwork_url': 'https://i1.sndcdn.com/artwor...,14
1,1174557856,Lofi Chill Playlist,https://i1.sndcdn.com/artworks-8ozxEOy5KjC9ZSH...,2020-12-07T02:02:17Z,NaN,6459236,all,NaN,playlist,NaN,...,NaN,https://api.soundcloud.com/playlists/1174557856,917161864,NaN,False,2020-12-07T02:02:17Z,2020-12-07T02:02:17Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,[{'artwork_url': 'https://i1.sndcdn.com/artwor...,39
2,1325679865,dramuha,NaN,2021-10-03T14:26:54Z,NaN,2080191,all,NaN,playlist,NaN,...,NaN,https://api.soundcloud.com/playlists/1325679865,917161870,NaN,False,2021-10-03T14:26:54Z,2021-10-03T14:26:54Z,{'avatar_url': 'https://a1.sndcdn.com/images/d...,[{'artwork_url': 'https://i1.sndcdn.com/artwor...,7
3,1174556389,niah playlist,NaN,2020-12-07T01:58:01Z,NaN,308116,all,NaN,playlist,NaN,...,NaN,https://api.soundcloud.com/playlists/1174556389,917161903,NaN,False,2020-12-07T01:58:01Z,2020-12-07T01:58:01Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,"[{'artwork_url': None, 'caption': None, 'comme...",1
4,1193047672,EMINEM,NaN,2021-01-11T15:23:37Z,NaN,3549533,all,NaN,playlist,NaN,...,NaN,https://api.soundcloud.com/playlists/1193047672,917161906,NaN,False,2021-01-11T15:23:37Z,2021-01-11T15:23:37Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,[{'artwork_url': 'https://i1.sndcdn.com/artwor...,14


In [7]:
df_playlist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 33 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                1001 non-null   int64  
 1   title             1001 non-null   object 
 2   artwork_url       23 non-null     object 
 3   created_at        1001 non-null   object 
 4   description       15 non-null     object 
 5   duration          1001 non-null   int64  
 6   embeddable_by     1001 non-null   object 
 7   genre             25 non-null     object 
 8   kind              1001 non-null   object 
 9   label_name        1 non-null      object 
 10  last_modified     1001 non-null   object 
 11  license           1001 non-null   object 
 12  likes_count       1001 non-null   int64  
 13  managed_by_feeds  1001 non-null   bool   
 14  permalink         1001 non-null   object 
 15  permalink_url     1001 non-null   object 
 16  public            1001 non-null   bool   


### 3. Thu thập dữ liệu "Track"

Tiến hành lấy dữ liệu tracks từ playlists.

In [8]:
## Request with trackid
# req_track = requests.get(f'https://api-v2.soundcloud.com/tracks/{trackid}?client_id={clientid}').json()
## Request with trackurl
# req_track = requests.get(f'https://api-v2.soundcloud.com/resolve?url={trackurl}&client_id={clientid}').json()

# Just to get the properties list
props_list = list(playlist_dict['tracks'][0][0].keys())

# Create an empty list with the props_list
track_dict = {}
for i in range(len(props_list)):
    track_dict[props_list[i]] = []

# Get ready to extract from playlists
count = 0

# Extract playlists until we got enough entries
print("Running, please wait...")
for playlist in playlist_dict['tracks']:
    if count >= num_of_entries:
        break
    for track in playlist: # By some reason, Soundcloud limit to 5 tracks here, no problem
        is_successful = True
        try:
            temp = track['title'] # Just to ensure that track is not null
            for i in range(len(props_list)):
                try:
                    track_dict[props_list[i]].append(track[props_list[i]])
                except:
                    track_dict[props_list[i]].append(None)
        except:
            is_successful = False
            break
        if is_successful:
            count += 1
            
# Just display for testing
df_track = pd.DataFrame(track_dict)
df_track.to_csv(os.path.join('./Api_data', 'track.csv'))

print(df_track.shape)
df_track.head()

Running, please wait...
(1001, 48)


,id,title,artwork_url,caption,commentable,comment_count,created_at,description,downloadable,download_count,...,visuals,waveform_url,display_date,media,station_urn,station_permalink,track_authorization,monetization_model,policy,user
0,226690288,"Rain Sounds - Sound Of Rain Nature Sounds, Rai...",https://i1.sndcdn.com/artworks-000175332551-ed...,NaN,True,771.0,2015-10-03T05:10:51Z,Sounds of rainfall with a cool space picture. ...,False,0.0,...,NaN,https://wave.sndcdn.com/E4c0Gu2NfHm6_m.json,2015-10-03T05:10:51Z,{'transcodings': [{'url': 'https://api-v2.soun...,soundcloud:system-playlists:track-stations:226...,track-stations:226690288,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,BLACKBOX,MONETIZE,{'avatar_url': 'https://i1.sndcdn.com/avatars-...
1,326671907,Ocean Waves 4 - Ocean Waves Natural White Nois...,https://i1.sndcdn.com/artworks-bZJG5aa0IZyZ-0-...,NaN,True,3.0,2017-06-06T16:50:10Z,NaN,False,0.0,...,NaN,https://wave.sndcdn.com/cRd1WjKS0uPz_m.json,2011-02-16T00:00:00Z,{'transcodings': [{'url': 'https://api-v2.soun...,soundcloud:system-playlists:track-stations:326...,track-stations:326671907,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,AD_SUPPORTED,MONETIZE,{'avatar_url': 'https://a1.sndcdn.com/images/d...
2,229953143,SOM DE CHUVA SOUND OF RAIN RELAX 30 MINUTES [B...,https://i1.sndcdn.com/artworks-000133852055-27...,NaN,True,900.0,2015-10-25T02:13:50Z,NaN,False,0.0,...,NaN,https://wave.sndcdn.com/GYyQoGYiXnwX_m.json,2015-10-25T02:13:50Z,{'transcodings': [{'url': 'https://api-v2.soun...,soundcloud:system-playlists:track-stations:229...,track-stations:229953143,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,NOT_APPLICABLE,ALLOW,{'avatar_url': 'https://i1.sndcdn.com/avatars-...
3,6768788,Sound of Rain,https://i1.sndcdn.com/artworks-000002938007-9e...,NaN,True,275.0,2010-11-07T01:36:05Z,Sleep better and relax with the sound of rain.,False,0.0,...,NaN,https://wave.sndcdn.com/fY87ipEt5bUp_m.json,2010-11-07T01:36:05Z,{'transcodings': [{'url': 'https://api-v2.soun...,soundcloud:system-playlists:track-stations:676...,track-stations:6768788,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,NOT_APPLICABLE,ALLOW,{'avatar_url': 'https://a1.sndcdn.com/images/d...
4,202304586,Relaxing Rain Sound (No Music),https://i1.sndcdn.com/artworks-000114447310-cn...,NaN,True,87.0,2015-04-24T07:15:34Z,Please set to loop in order to listen to it fo...,False,0.0,...,NaN,https://wave.sndcdn.com/pW05sMoEtMGS_m.json,2015-04-24T07:15:34Z,{'transcodings': [{'url': 'https://api-v2.soun...,soundcloud:system-playlists:track-stations:202...,track-stations:202304586,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,NOT_APPLICABLE,ALLOW,{'avatar_url': 'https://i1.sndcdn.com/avatars-...


In [9]:
df_track.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 48 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   1001 non-null   int64  
 1   title                1001 non-null   object 
 2   artwork_url          919 non-null    object 
 3   caption              22 non-null     object 
 4   commentable          1001 non-null   bool   
 5   comment_count        998 non-null    float64
 6   created_at           1001 non-null   object 
 7   description          448 non-null    object 
 8   downloadable         1001 non-null   bool   
 9   download_count       1000 non-null   float64
 10  duration             1001 non-null   int64  
 11  full_duration        1001 non-null   int64  
 12  embeddable_by        1001 non-null   object 
 13  genre                761 non-null    object 
 14  has_downloads_left   1001 non-null   bool   
 15  kind                 1001 non-null   o

---

## II. Thu thập dữ liệu bằng phương pháp parse HTML
Tổng quan: Thu thập hơn 1000 dữ liệu user, playlist và track trên trang **Soundcloud**

Phương pháp: Parse HTML

### Cài đặt chung cho parse HTML
* Biến `delay = 1`: đặt thời gian cho `driver` chờ mỗi khi thực hiện thao tác tìm kiếm (đơn vị: giây)
* Biến `op` để thiết lập các thuộc tính cho `driver`. Driver sử dụng thuộc tính `op.add_argument('headless')` để selenium không hiển thị browser mỗi khi thực hiện request.
* Driver được sử dụng ở phần này là Chrome Driver, với file thực thi **`chromedriver.exe`** đã được đặt chung folder với notebook này.

In [10]:
delay = 1
op = webdriver.ChromeOptions()
op.add_argument('headless')


### 1. Thu thập dữ liệu users


#### Hàm thực hiện crawl data
* Hàm `Get_User_Informations` có tham số truyền vào `url` là URL trang cá nhân của user.
* Hàm thực hiện parse HTML để lấy các thông tin sau:

|STT|Thông tin|Ý nghĩa|
| :---: | :------ | :--------------------- |
|1|ID|Mã ID của user|
|2|UserName|Tên của user|
|3|Verified|`TRUE` nếu tài khoản user đã được xác thực, `FALSE` nếu chưa xác thực|
|4|Full Name|Họ tên đầy đủ của user|
|5|Avatar URL|Link đến avatar của user|
|6|City|Thành phố user ở|
|7|Country|Đất nước user sống|
|8|Badge|Loại tài khoản user: `Pro Unlimited`,`Pro`. Tài khoản thường sẽ để trống|
|9|Description|Mô tả của user (có thể trống)|
|10|Followers Count|Số lượng người theo dõi user|
|11|Followings Count|Số lượng người mà user theo dõi|
|12|Tracks Count|Số lượng track của user|
|13|Likes Count|Số lượng track mà user đã like|
|14|Comments Count|Số lượng comment của user|
|15|Station URL|Link đến station của user|

* Thông tin của user được đưa vào list `users_info` (16 thông tin bao gồm cả URL của user)

In [11]:
def Get_User_Informations(url):
    # Lấy HTML
    driver = webdriver.Chrome(executable_path='./chromedriver.exe', options = op)
    driver.implicitly_wait(delay)
    driver.get(url)
    
    # 1. ID
    try:
        meta_line = driver.find_element_by_xpath("//meta[@property='al:ios:url']") # Tìm thẻ meta chứa userID
        content_attr = meta_line.get_attribute('content') # Lấy nội dung attribute 'content' có chứa userID
        userID = content_attr[19:] # Bỏ đi 19 ký tự đầu là được userID
    except:
        userID = ''
    # 2-3. UserName + Verified
    try:
        Username_line = driver.find_element_by_class_name('profileHeaderInfo__userName') # Tìm dòng chứa username
        userName = Username_line.text
        verified = 'False'
        if 'Verified' in userName: # username có dòng Verified thì đặt verified = true
            verified = 'True'
            userName = userName[:len(userName)-9] # cắt bỏ chuỗi verified trong username
    except:
        userName = ''
        verified = 'False'
    # 4. Full Name
    try:
        additional_Info = driver.find_element_by_class_name('profileHeaderInfo__additional') # Lấy dòng thông tin thêm
        fullName = ''
        if 'sc-mt-1x' not in additional_Info.get_attribute('class'): # Thẻ không có class = "sc-mt-1x" mới có thông tin Full Name
            fullName = additional_Info.text
    except:
        fullName = ''
    # 5. Avatar URL
    try:
        meta_line = driver.find_element_by_xpath("//meta[@property='og:image']") # Tìm thẻ meta chứa image url
        avatar_Url = meta_line.get_attribute('content') # Lấy nội dung attribute 'content' có chứa image url
    except:
        avatar_Url = ''
    # 6. City
    try:
        meta_line = driver.find_element_by_xpath("//meta[@property='og:locality']") # Tìm thẻ meta chứa thông tin locality
        city = meta_line.get_attribute('content') # Lấy nội dung attribute 'content' có chứa ctiy
    except:
        city = 'null'
    # 7. Country
    try:
        meta_line = driver.find_element_by_xpath("//meta[@property='og:country-name']") # Tìm thẻ meta chứa thông tin country-name
        country = meta_line.get_attribute('content') # Lấy nội dung attribute 'content' có chứa country
    except:
        country = 'undefined'
    # 8. Badge
    try:
        badge_Info = driver.find_element_by_class_name('creatorBadge') # Tìm dòng chứa thông tin badge
        badge = badge_Info.text
    except:
        badge = '' # Nếu không tìm được tức user ko có badge
    # 9. Description
    try:
        descript_line = driver.find_element_by_class_name('truncatedUserDescription') # Tìm đoạn chứa Description
        if 'm-overflow' in descript_line.get_attribute('class'): # Trường hợp Description chưa được hiển thị hết
            show_more = descript_line.find_element_by_class_name('truncatedUserDescription__collapse') # Tìm dòng Show more
            if show_more.text == 'Show more':
                driver.execute_script("arguments[0].click();", show_more) # Giả lập thao tác click Show more để hiển thị hết Description
        description = driver.find_element_by_class_name('truncatedUserDescription__content').text # Lấy thông tin Description
    except:
        description = ''
    # 10. Followers
    try:
        meta_line = driver.find_element_by_xpath("//meta[@property='soundcloud:follower_count']") # Tìm thẻ meta chứa số lượng follower
        followers = meta_line.get_attribute('content') # Lấy nội dung attribute 'content' có chứa số lượng follower
    except:
        followers = ''
    # 11. Followings
    try:
        x_path = "//*[@id='app']/div[2]/div[2]/div/div[4]/div[2]/div/*[@class='infoStats']/table/tbody/tr/td[2]/a"
        following_line = driver.find_element_by_xpath(x_path) # 
        followings = ''.join(filter(str.isdigit, following_line.get_attribute('title')))
    except:
        followings = 0
    # 12. Tracks
    try:
        meta_line = driver.find_element_by_xpath("//meta[@property='soundcloud:sound_count']") # Tìm thẻ meta chứa số lượng track
        tracks = meta_line.get_attribute('content') # Lấy nội dung attribute 'content' có chứa số lượng track
    except:
        tracks = ''
    # 13-14. Likes + Comments
    try:
        info = driver.find_element_by_class_name('userSidebar')
    except:
        likes = ''
        comments = ''
    try:
        like_info = info.find_element_by_class_name('likesModule')
        if 'empty' in like_info.get_attribute('class'):
            likes = ''
        else:
            likes = ''.join(filter(str.isdigit, like_info.find_element_by_class_name('sidebarHeader__actualTitle').text))
    except:
        likes = ''
    try:
        comment_info = info.find_element_by_class_name('commentsModule')
        if 'empty' in comment_info.get_attribute('class'):
            comments = ''
        else:
            comments = ''.join(filter(str.isdigit, comment_info.find_element_by_class_name('sidebarHeader__actualTitle').text))
    except:
        comments = ''
    # 15. Station URL
    try:
        info = driver.find_element_by_class_name('sc-button-startstation')
        station_Url = info.get_attribute('href')
    except:
        station_Url = ''
    
    # Đưa thông tin user vào list users_info
    users_info.append([userID, url, userName, fullName, avatar_Url, city, country, verified, badge, description, followers, followings, tracks, likes, comments, station_Url])
    # Đóng trang sau khi parse xong
    driver.close()

#### Hàm lấy URL của 1000 users
<a href="https://soundcloud.com/search/people?q=*">https://soundcloud.com/search/people?q=*</a> là trang web khi ta tìm kiếm `*` với *filter* là *people* trên **Soundcloud**. Kết quả hiển thị trên trang web là user bất kỳ của Soundcloud. Hàm `Search_Users` sẽ crawl trang web trên để lấy URL của >1000 user ngẫu nhiên.

In [12]:
def Search_Users():
    driver = webdriver.Chrome(executable_path='./chromedriver.exe', options = op)
    driver.implicitly_wait(delay)
    driver.get('https://soundcloud.com/search/people?q=*')
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # Scroll trang để load thêm dữ liệu
        items = driver.find_elements_by_class_name('userItem__coverArt')
        if (len(items)>1000): # Nếu load đủ >1000 dữ liệu thì dừng
            break
    for x in items: # Với mỗi url user tìm được
        Get_User_Informations(x.get_attribute('href')) # Gọi hàm để crawl data 
    # Đóng trang web
    driver.close()

#### Chương trình chính thu thập dữ liệu users
* Biến `users_info` là danh sách lưu thông tin các users. Mỗi 1 phần tử trong danh sách là 16 thông tin của 1 user
* Gọi hàm `Search_Users()` để tìm URL của >1000 users ngẫu nhiên.
* Với mỗi user tìm được, gọi hàm `Get_User_Informations` để lấy các thông tin của user đó (*thao tác này thực hiện bên trong hàm `Search_Users`*)
* Kết quả cuối cùng lưu trong biến `users_info` được ghi vào file `user.csv` trong thư mục `Crawl_data`

In [13]:
users_info = []
Search_Users()
df = pd.DataFrame(users_info, columns = ['ID', 'URL', 'Username', 'Full Name', 'Avatar URL', 'City', 'Country', 'Verified', 'Badge', 'Description', 'Followers Count', 'Followings Count', 'Tracks Count', 'Likes Count', 'Comments Count', 'Station URL'])
df.to_csv("./Crawl_data/user.csv", index = False, encoding = 'utf-8')

#### Kết quả thu thập dữ liệu user
* Tổng số mẫu thu thập được: **1010 records**.

* Thông tin 5 dòng đầu tiên của file *'user.csv'*.

In [14]:
res = pd.read_csv('./Crawl_data/user.csv')
res.head()

,ID,URL,Username,Full Name,Avatar URL,City,Country,Verified,Badge,Description,Followers Count,Followings Count,Tracks Count,Likes Count,Comments Count,Station URL
0,3340765,https://soundcloud.com/redstickmanagement,Red Stick Management,Brian Wolf,https://i1.sndcdn.com/avatars-000142100795-v8k...,Nashville,United States,False,NaN,NaN,2808907,0,0,NaN,NaN,NaN
1,3945382,https://soundcloud.com/deutschegrammophon,Deutsche Grammophon,Deutsche Grammophon,https://i1.sndcdn.com/avatars-000248442991-gwv...,Berlin,Germany,True,NaN,Welcome to Deutsche Grammophon!\nRecording gre...,2014123,48,0,14.0,NaN,NaN
2,6546602,https://soundcloud.com/jimmycliffmusic,Jimmy Cliff,Jimmy Cliff,https://i1.sndcdn.com/avatars-000005932628-vxl...,NaN,undefined,False,NaN,NaN,2809376,0,2,NaN,NaN,https://soundcloud.com/discover/sets/artist-st...
3,67953725,https://soundcloud.com/mustafahosny,Mustafa Hosny,Mustafa Hosny,https://i1.sndcdn.com/avatars-kGR5sxPnwZVrh7E6...,Cairo,Egypt,False,Pro Unlimited,الحساب الرسمي لأرشيف الصوتيات الخاص بالداعية م...,97374,0,5054,NaN,NaN,https://soundcloud.com/discover/sets/artist-st...
4,2461663,https://soundcloud.com/intelligence2,Intelligence Squared,Intelligence Squared,https://i1.sndcdn.com/avatars-000641566545-k2s...,London,undefined,False,NaN,Intelligence Squared is the world’s leading fo...,1297852,2,1,1.0,NaN,https://soundcloud.com/discover/sets/artist-st...


* Thông tin chung về các mẫu đã thu thập.

In [15]:
res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010 entries, 0 to 1009
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                1010 non-null   int64  
 1   URL               1010 non-null   object 
 2   Username          1010 non-null   object 
 3   Full Name         635 non-null    object 
 4   Avatar URL        968 non-null    object 
 5   City              663 non-null    object 
 6   Country           1010 non-null   object 
 7   Verified          1010 non-null   bool   
 8   Badge             448 non-null    object 
 9   Description       647 non-null    object 
 10  Followers Count   1010 non-null   int64  
 11  Followings Count  1010 non-null   int64  
 12  Tracks Count      1010 non-null   int64  
 13  Likes Count       701 non-null    float64
 14  Comments Count    538 non-null    float64
 15  Station URL       941 non-null    object 
dtypes: bool(1), float64(2), int64(4), object(9


### 2. Thu thập dữ liệu playlists


Tạo `driver` với các thuộc tính options được thiết lập trong biến `op` ở trên.

In [16]:
driver = webdriver.Chrome(executable_path='./chromedriver.exe', options = op)

#### Crawl URL về file
Ta cần lấy danh sách các URL cần thiết để thực hiện request ở phần sau bằng cách sử dụng file `user.csv` để lấy các url từ mỗi user.

Từ list words ta thực hiện tìm kiếm các từ khóa sau đó nhận về kết quả. Lưu vào file 'url.txt' (ở đây giới hạn chỉ lấy 1200 kết quả).

In [17]:
%%time
df = pd.read_csv('./Crawl_data/user.csv')
urls = list(df.URL)
playlist_urls = []
numLink = 1200
for url in urls:
    try:
        driver.get(url+'/sets')
        WebDriverWait(driver, 3, poll_frequency=1).until(EC.presence_of_all_elements_located((By.XPATH, "//*[@class='soundList lazyLoadingList']")))
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        main = soup.find('div', {'class': 'userMain'})

        pls = main.find_all('a', {'class': 'soundTitle__title'})
        for pl in pls:
            playlist_urls.append('https://soundcloud.com/'+pl.get('href'))

        if len(playlist_urls)>=numLink:
            break
    except Exception:
        continue
playlist_urls = list(set(playlist_urls))
textfile = open("url.txt", "w")
for link in playlist_urls:
    textfile.write(link+'\n')
textfile.close()

Wall time: 12min 56s


### Hàm thực hiện crawl data
Tham số truyền vào là `FileName`. Thực hiện lấy kết quả từ file `url.txt` để request và lấy các thuộc tính.

In [18]:
def readFileURL(FileName):   
    with open(FileName, "r") as f:
        url=[]
        for x in f:
          url.append(x)
    return url

In [19]:
def crawlData(FileName):
    lines=[]
    # Gọi hàm đọc URL đã lấy từ bước trước
    urls=readFileURL(FileName)
    pattern=r'"id": ([0-9]+), "kind": "track"'
    for url in urls:
        try: 
            driver.get(url)
            WebDriverWait(driver, 3, poll_frequency=1).until(EC.presence_of_all_elements_located((By.XPATH, "//body[@class='sc-classic']//script")))
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            getScript=soup.find("body",{'class':'sc-classic'})
            script=json.loads(str(getScript.find_all('script')[5].string.replace('window.__sc_hydration = ','')[:-1]))     
            lineData=script[6]['data']
            lineStr=json.dumps(lineData)
            line=[]
            # Thực hiện lấy id của playlist
            id=lineData['id']
            line.append(id)
            # Thực hiện lấy username
            username=soup.find("h2",{"class":"soundTitle__username"}).select_one("a").text.strip()
            line.append(username)
            # Thực hiện lấy tên playlist
            title=soup.find("h1",{"class":"soundTitle__title"}).find('span').text
            line.append(title)
            #Vòng lặp thực hiện lấy các thuộc tính trong fields_script từ script
            for field in fields_script:
                if field != 'tracks':
                    line.append(lineData[field])
                    continue
                # Nếu thuộc tính là tracks thì chỉ thực hiện lấy id của mỗi track
                trackId=','.join(re.findall(pattern,lineStr))
                line.append(trackId)
            # Thực hiện lấy số lượng track trong playlist 
            countTrack=soup.find("div",{"class":"genericTrackCount__title"}).text
            line.append(countTrack)
            #Thực hiện lấy thuộc tính duration  
            duration=soup.find("div",{"class":"genericTrackCount__duration"}).text
            line.append(duration)
            # Thực hiện lấy các thuộc tính liên quan thời gian
            published_datetime=soup.find("time",{"class":"relativeTime"}).get('datetime')
            line.append(published_datetime)
            published_time=soup.find("time",{"class":"relativeTime"}).find_all('span')[1].text
            line.append(published_time)
            lines.append(line)
        except Exception:
            continue
    return lines

### Thực hiện gọi hàm và lưu kết quả
Gọi hàm `crawlData` và lưu kết quả vào biến `lines`, sau đó thực hiện lưu kết quả vào file `playlist.csv` trong folder `Crawl_data`

In [20]:
def writeCSV(FileName,lines):
    #tạo biến cols chứa tên các thuộc tính sẽ được lưu vào file 
    cols=['id','username','title']
    column=['countTrack','duration','published_datetime','published_time']
    cols.extend(fields_script)
    cols.extend(column)
    
    out=pd.DataFrame(lines,columns=cols)
    out.to_csv(FileName, index=False,encoding='utf-8')

In [21]:
%%time 
InFile='url.txt'
Outfile='./Crawl_data/playlist.csv'
# fields_script lưu những thuộc tính sẽ được lấy từ script
fields_script =['artwork_url', 'description', 'last_modified', 'likes_count', 'permalink_url','reposts_count', 'sharing', 'uri', 'user_id', 'tracks']
lines=crawlData(InFile)
writeCSV(Outfile,lines)

Wall time: 33min 2s


In [22]:
driver.close()

Thông tin 5 dòng đầu tiên của file 'playlist.csv'

In [24]:
res = pd.read_csv('./Crawl_data/playlist.csv')
res.head()

,id,username,title,artwork_url,description,last_modified,likes_count,permalink_url,reposts_count,sharing,uri,user_id,tracks,countTrack,duration,published_datetime,published_time
0,395302475,MTV,MTV Clubland: Remix & Chill,https://i1.sndcdn.com/artworks-000268334123-gj...,featured artist: shallou (@shallou)\nartwork &...,2017-12-11T19:19:22Z,60,https://soundcloud.com/mtvofficial/sets/mtv-cl...,8,public,https://api.soundcloud.com/playlists/395302475,3002666,"350036662,357157187,362608991,362652107,352987...",29,1:42:31,2017-12-11T19:19:20.000Z,3 years ago
1,1138252063,Success Program,Nivel 18 - Unit number Four,NaN,NaN,2020-10-01T22:53:07Z,0,https://soundcloud.com/canal-personal-sucess/s...,0,public,https://api.soundcloud.com/playlists/1138252063,870506323,"903101224,903101218,903101206,903101197,903101...",48,1:06:14,2020-10-01T22:53:07.000Z,1 year ago
2,551270061,LA Riots,Guest DJ Mixes,NaN,NaN,2018-06-29T19:58:04Z,3,https://soundcloud.com/lariots/sets/guest-dj-m...,0,public,https://api.soundcloud.com/playlists/551270061,345353,"415767717,393533937,369038876,297411749",4,4:30:27,2018-06-29T19:58:04.000Z,3 years ago
3,8428248,Pleasance,Pleasance Comedy Podcast 2012,https://i1.sndcdn.com/artworks-000054191356-1k...,A collection of our 2012 Pleasance Comedy Podc...,2014-08-01T08:58:02Z,14,https://soundcloud.com/thepleasance/sets/pleas...,3,public,https://api.soundcloud.com/playlists/8428248,6278531,NaN,0,8:14:40,2013-07-31T11:30:31.000Z,8 years ago
4,104712463,user850053910,GA interim assessment,NaN,NaN,2015-05-05T14:04:08Z,0,https://soundcloud.com/missharris/sets/ga-inte...,0,public,https://api.soundcloud.com/playlists/104712463,9751632,NaN,0,7:54,2015-05-05T14:04:08.000Z,6 years ago


Thông tin chung về các mẫu đã thu thập.

In [25]:
res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  1050 non-null   int64 
 1   username            1050 non-null   object
 2   title               1050 non-null   object
 3   artwork_url         534 non-null    object
 4   description         323 non-null    object
 5   last_modified       1050 non-null   object
 6   likes_count         1050 non-null   int64 
 7   permalink_url       1050 non-null   object
 8   reposts_count       1050 non-null   int64 
 9   sharing             1050 non-null   object
 10  uri                 1050 non-null   object
 11  user_id             1050 non-null   int64 
 12  tracks              773 non-null    object
 13  countTrack          1050 non-null   int64 
 14  duration            1050 non-null   object
 15  published_datetime  1050 non-null   object
 16  published_time      1050

### Nhận xét
Có tổng cộng 17 cột với 1050 mẫu kết quả thu thập được. Mỗi dòng thể hiện các thuộc tính của playlist bao gồm: id của playlist, username (tên chủ playlist), title (tên playlist), artwork_url (link đến ảnh của playlist), description (mô tả), last_modified, likes_count (số lượt like), permalink_url (link playlist), reposts_count (số lượng repost), sharing (lượt share), uri, user_id (id người dùng), tracks (id của track trong playlist), countTrack (tính số lượng track), duration (thời lượng), published_datetime, published_time (thời gian đăng). Đa phần thông tin lấy từ phần header trả về, các thông tin còn lại được lấy trực tiếp từ phần dữ liệu có khi mở trang web.


### 3. Thu thập dữ liệu tracks

Các dữ liệu của tracks sẽ được lấy sau khi có dữ liệu của playlists (track sẽ lấy từ các playlist tìm được).

Hàm thực hiện việc crawl data với tham số truyền vào là `driver`,`in_playlist_url`. Các thuộc tính được lấy bằng các hàm có sẵn của selenium (`find_element_by_css_selector`,`find_element_by_xpath`) và trả về dưới dạng 1 list.

In [26]:
def crawlDatas(driver,in_playlist_url):
    #1. Thực hiện việc lấy link gốc của track (track không nằm trong playlist)
    try:
        default_url=driver.find_element_by_css_selector('link[rel="canonical"]').get_attribute('href')
    except:
        default_url=''
    #2. Lấy tựa đề của track
    try:
        title=driver.find_element_by_css_selector("h1.soundTitle__title").text
    except:
        title=""
    #3. Lấy tên tác giả của track và link đến trang của tác giả
    try:
        un=driver.find_element_by_css_selector("h2.soundTitle__username a")
        username=un.text.strip()
        username_link=un.get_attribute('href')
    except:
        username=""
        username_link=''
    #4. Lấy playlist đang chứa track (vì các bài hát được lấy đều nằm trong playlist)
    try:
        in_playlist=driver.find_element_by_css_selector('span.inPlaylist__title').text
    except:
        in_playlist=''
    #5. Lấy poster của track
    try:
        artwork_url=driver.find_element_by_css_selector("span.sc-artwork").value_of_css_property("background-image").split('"')[1]
    except:
        artwork_url=""
    #6. Lấy thời gian tác giả khởi tạo track
    try:
        create_time=driver.find_element_by_css_selector("time.relativeTime").get_attribute("datetime")
    except:
        create_time=""
    #7. Lấy tên của người sở hữu playlist chứa track và link đến trang của người đó (vì track được lấy từ playlist)
    try:
        unp=driver.find_element_by_css_selector('h3.userBadge__username a')
        username_playlist=unp.text.strip()
        username_playlist_url=unp.get_attribute('href')
    except:
        username_playlist=''
        username_playlist_url=''
    #8. Lấy số lượng lượt phát, like và repost
    try:
        sound_Stats=driver.find_element_by_css_selector('ul.soundStats')
        ss = [''.join(filter(str.isdigit,x.get_attribute("title"))) for x in sound_Stats.find_elements_by_tag_name("li")]
        playback_count=ss[0]
        likes_count=ss[1]
        reposts_count=ss[2]
    except:
        playback_count=""
        likes_count=""
        reposts_count=""
    #9. Lấy số lượng comment
    try:
        cmt=driver.find_element_by_css_selector("span.commentsList__actualTitle")
        comment_count=''.join(filter(str.isdigit,cmt.text))
    except:
        comment_count=''
    #10. Lấy độ dài track
    try:
        duration=driver.find_element_by_xpath('//*[@id="app"]/div[4]/section/div/div[3]/div[3]/div/div[3]/span[2]').text
    except:
        duration=''
    #11. Lấy thông tin mua track (có phí hay không và đường link đến nơi mua, thông tin này có thể có hoặc không)
    try:
        purchase=driver.find_element_by_css_selector("a.soundActions__purchaseLink")
        purchase_title=purchase.text
        purchase_url=purchase.get_attribute('title')
    except:
        purchase_title=''
        purchase_url=''
    #12. Lấy mô tả về track
    try:
        description=driver.find_element_by_css_selector("div.truncatedAudioInfo__content div.sc-text-body").text.strip()
    except:
        description=''
    #13. Lấy id của track
    try:
        tid=driver.find_element_by_css_selector('meta[content^="soundcloud://sounds:"]').get_attribute("content")
        track_id=''.join(filter(str.isdigit,tid))
    except:
        track_id=''
    #14. Lấy thể loại của track (có thể có hoặc không tùy vào track)
    try:
        genre=driver.find_element_by_css_selector("span.sc-tagContent").text
    except:
        genre=''
    #15. Lấy danh sách các tag (có thể có hoặc không tùy vào track)
    try:
        tag_list=driver.find_element_by_css_selector("div.soundTags").text.strip().replace("\n"," ")
    except:
        tag_list=''
    #16. Lấy thông tin các track liên quan (tên track)
    try:
        rt=driver.find_elements_by_css_selector('article.relatedSoundsModule a.soundTitle__title')
        related_track=[t.get_attribute('title') for t in rt]
        related_track=';'.join(related_track)
    except:
        related_track=''
    # Trả về list các thông tin thu được từ 1 track
    return [track_id,in_playlist_url,default_url,title,username,
               username_link,in_playlist,artwork_url,create_time,username_playlist,username_playlist_url,
               description,playback_count,likes_count,reposts_count,comment_count,
               duration,purchase_title,purchase_url,genre,tag_list,related_track]

Thực hiện đọc nội dung từ file *playlist.csv*. Thông tin thu được sẽ được lưu vào biến `columms`.

In [27]:
columns = defaultdict(list)
with open('./Crawl_data/playlist.csv',encoding='utf-8') as f:
    reader = csv.DictReader(f) 
    for row in reader:
        for (k,v) in row.items():
            columns[k].append(v)

Tiến hành khởi tạo `driver` với thuộc tính `options` đã được cài đặt ở bên trên. Hàm `driver.implicitly_wait(delay)` sẽ đợi cho đến khi tìm thấy với thời gian tối đa `delay=1` mỗi khi thực hiện tìm kiếm một thuộc tính.

In [28]:
driver = webdriver.Chrome(options=op)
driver.implicitly_wait(delay)

Lấy danh sách các *url* cần thiết để thực hiện *request* ở phần sau. Các **track** được lấy từ **playlist** đã thu được từ file **playlist.csv**. Do các track được thiết kế để lấy từ **playlist** nên ở thao tác này ta sẽ lấy các *playlist url* để thực hiện *request*. Với mỗi *playlist url* ta sẽ tìm tất cả các *track url* có trong *playlist* đó và lưu vào biến `trackList`. Vòng lặp dừng lại đến khi có hơn *1000 url* trong `trackList`. 

In [29]:
%%time
# Khởi tạo trackList và biến đếm count để đếm thứ tự playlist.
trackList=[]
count=0
# Vòng lặp để lấy url của track, dừng lại khi số url lớn hơn 1000
while len(trackList)<=1000:
    driver.get(columns['permalink_url'][count])
    tracks=driver.find_elements_by_css_selector('a.trackItem__trackTitle')
    tracks=[tr.get_attribute('href') for tr in tracks]
    trackList+=tracks
    count+=1

Wall time: 10min 8s


Thực hiện request *track* từ các *url* đã lấy được và ghi vào file ***track.csv***.

In [30]:
%%time
with open("./Crawl_data/track.csv", mode='w',encoding="utf-8",newline='') as fileout:
    # Ghi tên các cột
    header=['track_id','in_playlist_url','default_url','title','username',
            'username_link','in_playlist','artwork_url','create_time','username_playlist','username_playlist_url',
            'description','playback_count','likes_count','reposts_count','comment_count',
            'duration','purchase_title','purchase_url','genre','tag_list','related_track']
    response = csv.writer(fileout, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    response.writerow(header)
    # Thực hiện request và ghi kết quả trả về từ hàm crawlDatas vào file
    for url in trackList:
        driver.get(url)
        response.writerow(crawlDatas(driver,url))

Wall time: 1h 24min 22s


Kết thúc `driver`.

In [31]:
driver.quit()

Thông tin 5 dòng đầu tiên của file *tracks.csv*.

In [32]:
res = pd.read_csv('./Crawl_data/track.csv')
res.head()

,track_id,in_playlist_url,default_url,title,username,username_link,in_playlist,artwork_url,create_time,username_playlist,...,playback_count,likes_count,reposts_count,comment_count,duration,purchase_title,purchase_url,genre,tag_list,related_track
0,350036662,https://soundcloud.com/shallou/you-and-me?in=m...,https://soundcloud.com/shallou/you-and-me,You and Me,shallou,https://soundcloud.com/shallou,MTV Clubland: Remix & Chill,https://i1.sndcdn.com/artworks-000250215235-hy...,2017-11-02T07:41:37.000Z,MTV,...,1537266.0,28386.0,3737.0,269.0,3:54,Buy / Stream,http://www.shallou.lnk.to/souls,Electronic,NaN,Only;In The End (ft. Akacia);Lost In Space
1,357157187,https://soundcloud.com/crymynals/crazy-alive-f...,https://soundcloud.com/crymynals/crazy-alive-f...,Crazy & Alive (feat. Choice),Crymynals,https://soundcloud.com/crymynals,MTV Clubland: Remix & Chill,https://i1.sndcdn.com/artworks-000257623415-36...,2017-11-17T16:19:35.000Z,MTV,...,160779.0,3752.0,657.0,43.0,3:53,Free Download,https://www.hive.co/downloads/download/501070/...,Electronic,Soul Future Bass,Two Friends ft. Kevin Writer - While We're Dre...
2,362608991,https://soundcloud.com/elementthecity/what-lov...,https://soundcloud.com/elementthecity/what-lov...,Maroon 5 - What Lovers Do ft. SZA (Element Remix),Element,https://soundcloud.com/elementthecity,MTV Clubland: Remix & Chill,https://i1.sndcdn.com/artworks-000262832738-54...,2017-11-29T19:01:41.000Z,MTV,...,202088.0,3456.0,702.0,60.0,3:05,free download,https://click.dj/elementthecity/maroon-5-what-...,Electronic,NaN,Louis The Child - Better Not feat. Wafia (KRAN...
3,362652107,https://soundcloud.com/offramp/post-malone-con...,https://soundcloud.com/offramp/post-malone-con...,Post Malone - Congratulations (Offramp Remix),Offramp,https://soundcloud.com/offramp,MTV Clubland: Remix & Chill,https://i1.sndcdn.com/artworks-000262874978-oo...,2017-11-29T20:41:48.000Z,MTV,...,35746.0,567.0,72.0,14.0,3:26,Download,https://theartistunion.com/tracks/9d5561,NaN,NaN,Hibshi Ft. AiMEE - Cold Beer;The Chainsmoker...
4,352987598,https://soundcloud.com/porter-robinson/virtual...,https://soundcloud.com/porter-robinson/virtual...,Virtual Self - Ghost Voices,Porter Robinson,https://soundcloud.com/porter-robinson,MTV Clubland: Remix & Chill,https://i1.sndcdn.com/artworks-000253103615-cj...,2017-11-08T20:00:54.000Z,MTV,...,3956493.0,85290.0,12036.0,1101.0,4:26,Watch Video,http://youtu.be/HPc8QMycGno,NEOTRANCE,NaN,Tell Me;Virtual Self - a.i.ngel (Become God) [...


Thông tin chung về các mẫu đã thu thập.

In [33]:
res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   track_id               1001 non-null   int64  
 1   in_playlist_url        1001 non-null   object 
 2   default_url            1001 non-null   object 
 3   title                  988 non-null    object 
 4   username               997 non-null    object 
 5   username_link          998 non-null    object 
 6   in_playlist            1000 non-null   object 
 7   artwork_url            990 non-null    object 
 8   create_time            1001 non-null   object 
 9   username_playlist      988 non-null    object 
 10  username_playlist_url  988 non-null    object 
 11  description            595 non-null    object 
 12  playback_count         426 non-null    float64
 13  likes_count            426 non-null    float64
 14  reposts_count          426 non-null    float64
 15  comm

## Nhận xét
Có tổng cộng 22 cột với 1000 mẫu kết quả thu thập được sau khi crawl. Mỗi cột thể hiện một thuộc tính của *track* theo thứ tự bao gồm: *id, link track từ playlist, link track khi không trong playlist, tên track, tên tác giả, link đến trang của tác giả, tên playlist chứa track, link đến poster của track, thời gian khởi tạo track, tên của người sở hữu playlist chứa track, link đến trang của người sở hữu playlist chứa track, mô tả của track, số lượt phát lại, số lượt like, số lượt repost, số lượt comment, độ dài track, thông tin mua track, link đến nơi mua track, thể loại track, danh sách tag và các track liên quan*. Ngoại trừ các thông tin như *id, url* được lấy từ phần header trả về, các thông tin còn lại đều được lấy trực tiếp từ phần dữ liệu có thể nhìn thấy khi mở trang web. Những cột dữ liệu có thể có giá trị `NaN` như *description, purchase_title, purchase_url, genre, tag_list,..* có thể bắt nguồn từ 1 trong các lí do sau:
* Selenium không lấy được thông tin trong thời gian chờ tối đa 1s.
* Người đăng tải hoặc người sở hữu track đã không thêm vào các thông tin đó khi đăng tải.
* Track không có sẵn ở thị trường Việt Nam.
* ...


# III. Nguồn tham khảo

[Document của Soundclound](https://developers.soundcloud.com/)

[Selenium with Python](https://selenium-python.readthedocs.io/)